#### I establish the baseline group metrics for adopters and non adopters. In order to determine the impact of the release, I must establish a baseline so that metrics can be compared **relative to** a baseline

In [15]:
import pandas as pd
import os
from pathlib import Path
import duckdb

base_dir = Path('c:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Revenue-Sustainability-Analysis')
data_dir = Path(base_dir / 'Dataset')

# feats incudes all feature_usage history both pre and post release
feats = pd.read_parquet(data_dir / 'feature_usage.parquet')

pd.set_option('display.max_rows',10)

### Partition users into 3 tiered groups: 
* Non-Users (0+ distinct usage days)
* Experimenters (1-2 distinct usage days)
* Adopters (3+ distinct usage days)

First, I want to filter to only those who have any feature_usage history pre release

In [16]:
# Normalize the grain of the table to be one row per sub_id, usage_id
feats = feats.groupby(['subscription_id', 'usage_date', 'feature_name'], as_index=False) \
   .agg({
        'usage_count': 'sum',
        'usage_duration_secs': 'sum',
        'error_count': 'sum',
        'is_beta_feature': 'max'
   })

feats['usage_date'] = pd.to_datetime(feats['usage_date'])

## Validation

# feats[feats.duplicated(subset=['subscription_id', 'usage_date'], keep=False)] \
#             .sort_values(by=['subscription_id', 'usage_date']) \
#             .dropna(subset=['subscription_id', 'usage_date'])

# feats.loc[feats['subscription_id'] == 'S-012ab9'].sort_values(by='usage_date')

### Partition users into 3 tiered groups: 
* Non-Users (0+ distinct usage days)
* Experimenters (1-2 distinct usage days)
* Adopters (3+ distinct usage days)

### Partition users into 3 tiered groups: 
* Non-Users (0+ distinct usage days)
* Experimenters (1-2 distinct usage days)
* Adopters (3+ distinct usage days)

In [17]:
# Pre has all information for both adopters and non adopters 90 days before the release date
# Pre is at the feature usage event level
pre = feats.loc[(feats['usage_date'] < '2023-06-12')].copy()

pre_ids = pre['subscription_id'].dropna().unique()

pre_window = pre.loc[pre['usage_date'] >= '2023-03-14']

### Calculate pre-release metrics at the customer level

Define pre-release window as 90 days prior to June 16, 2023 => March 14, 2023

In [18]:
# Count how many unique days each person used the platform
distinct_usage_days = pre_window.groupby('subscription_id')['usage_date'] \
          .nunique() \
          .reset_index(name='pre_distinct_usage_days')

# Calculate avg gaps between days
# pre_window has the feature_level of the pre release window
pre_window = pre_window.sort_values(['subscription_id', 'usage_date'])
pre_window['gaps'] = pre_window.groupby('subscription_id')['usage_date'] \
    .diff() \
    .dt.days \
    .astype('Int64')

avg_gaps = pre_window.groupby('subscription_id')['gaps'] \
            .mean() \
            .reset_index(name='pre_avg_gaps')

# pre_metrics has the subscription_id level metrics of the pre release window 
pre_metrics = distinct_usage_days.merge(
            avg_gaps,
            on="subscription_id",
            how='outer'
)

# Calculate total usage and error counts for each subscription_id
total_usage = pre_window.groupby('subscription_id')['usage_count'].sum() \
                .reset_index(name='pre_total_usage')

total_usage_duration = pre_window.groupby('subscription_id')['usage_duration_secs'].sum() \
                .reset_index(name='pre_total_usage_duration')

total_error_count = pre_window.groupby('subscription_id')['usage_duration_secs'].sum() \
                .reset_index(name='pre_total_error_count')

# Cust is aggregated at the customer level, still has both adopters and non adopters
pre_metrics = pre_metrics.merge(
        total_usage,
        on='subscription_id',
        how='left'
)

pre_metrics = pre_metrics.merge(
        total_usage_duration,
        on='subscription_id',
        how='left'
)

pre_metrics = pre_metrics.merge(
        total_error_count,
        on='subscription_id',
        how='left'
)

# Calcuate daily averages
pre_metrics['pre_avg_daily_usage'] = (pre_metrics['pre_total_usage'] / pre_metrics['pre_distinct_usage_days']).round(2)
pre_metrics['pre_avg_daily_usage_duration'] = (pre_metrics['pre_total_usage_duration'] / pre_metrics['pre_distinct_usage_days']).round(2)
pre_metrics['pre_avg_daily_error_count'] = (pre_metrics['pre_total_error_count'] / pre_metrics['pre_distinct_usage_days']).round(2)

# For those who are not within the pre release window, fill 0 as inactive behavior 
pre_metrics[['pre_distinct_usage_days', 
             'pre_total_usage', 
             'pre_total_usage_duration',
             'pre_total_error_count',
             'pre_avg_gaps', 
             'pre_avg_daily_usage', 
             'pre_avg_daily_usage_duration', 
             'pre_avg_daily_error_count'
            ]] = pre_metrics[['pre_distinct_usage_days', 
                                'pre_total_usage', 
                                'pre_total_usage_duration',
                                'pre_total_error_count',
                                'pre_avg_gaps', 
                                'pre_avg_daily_usage', 
                                'pre_avg_daily_usage_duration', 
                                'pre_avg_daily_error_count'
            ]].fillna(0)


# Include the subscription_ids who do not have a history within the 90 day pre release window
pre_metrics = pre_metrics.merge(pd.DataFrame({'subscription_id': pre_ids}), on='subscription_id', how='left')

# Create tiers for each customer based on their pre release information
# Partition customers based on distinct_usage_days
pre_metrics['tier'] = pd.cut(
    pre_metrics['pre_distinct_usage_days'],
    bins=[-1,1,3, pre_metrics['pre_distinct_usage_days'].max()],
    labels=['Tier 3 (Low)', 'Tier 2 (Med)', 'Tier 1 (High)']
)

## Validation
# cust.groupby('tier')['distinct_usage_days'].describe()
# cust['tier'].value_counts

print(f"Check for duplicates: {pre_metrics.duplicated(subset='subscription_id').any()}")

Check for duplicates: False


### Partition users into 3 tiered groups: 
* Non-Users (0+ distinct usage days)
* Experimenters (1-2 distinct usage days)
* Adopters (3+ distinct usage days)

### Calculate Post metrics for the short term

In [19]:
# Filters feature history to all feature usage after the release date
post_feat_history = feats.loc[feats['usage_date'] >= '2023-06-12']

# Feature-level table of short term window
post_short_window = post_feat_history.loc[post_feat_history['usage_date'] < '2023-09-10']

# Filter to only those who interacted with product but only to determine adoption_flag
post_window_newai = post_short_window.loc[post_short_window['feature_name'] == 'feature_newai']

# Define adoption_flag for each subscription id. Adoption of feature = 2+ distinct usage dates
# Count how many distinct usage days for each user 
post_usage_ai = post_window_newai.groupby('subscription_id')['usage_date'] \
                                                   .nunique() \
                                                   .reset_index(name='post_distinct_usage_days')

post_usage_ai['adoption_flag'] = (post_usage_ai['post_distinct_usage_days'] >= 2).astype('Int64')

# ~20% of users of the feature are adopters
post_usage_ai['adoption_flag'].mean()

0.19401993355481728

In [ ]:
# Calculate Avg gaps post release
post_short_window = post_short_window.sort_values(by=['subscription_id','usage_date'])
post_short_window['gaps'] = post_short_window.groupby('subscription_id')['usage_date']\
                                             .diff() \
                                             .dt.days \
                                             .astype('Int64')

avg_gaps = post_short_window.groupby('subscription_id')['gaps'] \
            .mean() \
            .reset_index(name='short_avg_gaps')

In [ ]:
# Calculate usage metrics: total_usage, total_usage_duration, avg_daily_usage, avg_daily_usage_duration
total_usage = post_short_window.groupby('subscription_id')['usage_count'].sum() \
                               .reset_index(name='short_total_usage')

total_usage_duration = post_short_window.groupby('subscription_id')['usage_duration_secs'].sum() \
                                        .reset_index(name='short_total_usage_duration')

total_error_count = post_short_window.groupby('subscription_id')['error_count'].sum() \
                                        .reset_index(name='short_total_error_count')

post_metrics = post_usage_ai.merge(avg_gaps, on='subscription_id', how='left')
post_metrics = post_metrics.merge(total_usage, on='subscription_id', how='left') 
post_metrics = post_metrics.merge(total_usage_duration, on='subscription_id', how='left')
post_metrics = post_metrics.merge(total_error_count, on='subscription_id', how='left')

# Calculate averages
post_metrics['post_avg_daily_usage'] = (post_metrics['short_total_usage'] / post_metrics['short_distinct_usage_days']).round(2)
post_metrics['post_avg_daily_usage_duration'] = (post_metrics['short_total_usage_duration'] / post_metrics['post_distinct_usage_days']).round(2)
post_metrics['post_avg_daily_error_count'] = (post_metrics['post_total_error_count'] / post_metrics['post_distinct_usage_days']).round(2)

# Check for dupes, ensure post_metrics is the subscription_id level
print(f"Check for duplicates: {post_metrics.duplicated(subset='subscription_id').any()}")

subs = pre_metrics.merge(post_metrics, on='subscription_id', how='left')

Check for duplicates: False


In [22]:
subs['tier'].unique()

['Tier 2 (Med)', 'Tier 3 (Low)', 'Tier 1 (High)']
Categories (3, object): ['Tier 3 (Low)' < 'Tier 2 (Med)' < 'Tier 1 (High)']

In [23]:
tier1 = subs.loc[subs['tier'] == 'Tier 1 (High)']
tier2 = subs.loc[subs['tier'] == 'Tier 2 (Med)']
tier3 = subs.loc[subs['tier'] == 'Tier 3 (Low)']

In [ ]:
t1_short_metrics = tier1.groupby('adoption_flag').agg({
                        'post_distinct_usage_days': 'mean',
                        'post_avg_gaps': 'mean',
                        'post_total_usage': 'sum',
                        'post_total_usage_duration': 'sum',
                        'post_total_error_count': 'sum',
                        'post_avg_daily_usage': 'mean',
                        'post_avg_daily_usage_duration': 'mean',
                        'post_avg_daily_error_count': 'mean'})

t2_short_metrics = tier2.groupby('adoption_flag').agg({
                        'post_distinct_usage_days': 'mean',
                        'post_avg_gaps': 'mean',
                        'post_total_usage': 'sum',
                        'post_total_usage_duration': 'sum',
                        'post_total_error_count': 'sum',
                        'post_avg_daily_usage': 'mean',
                        'post_avg_daily_usage_duration': 'mean',
                        'post_avg_daily_error_count': 'mean'})

t3_short_metrics = tier3.groupby('adoption_flag').agg({
                        'post_distinct_usage_days': 'mean',
                        'post_avg_gaps': 'mean',
                        'post_total_usage': 'sum',
                        'post_total_usage_duration': 'sum',
                        'post_total_error_count': 'sum',
                        'post_avg_daily_usage': 'mean',
                        'post_avg_daily_usage_duration': 'mean',
                        'post_avg_daily_error_count': 'mean'})

,post_distinct_usage_days,post_avg_gaps,post_total_usage,post_total_usage_duration,post_total_error_count,post_avg_daily_usage,post_avg_daily_usage_duration,post_avg_daily_error_count
adoption_flag,,,,,,,,
0,1.0,25.180314,4188,1215513,243,18.051724,5239.280172,1.047414
1,2.2,24.102444,2137,660056,137,13.108667,3983.646533,0.849067


#### Calculate post long term metrics. The window is from 2023-09-10 to present 

In [28]:
post_long_window = post_feat_history.loc[post_feat_history['usage_date'] > '2023-09-10']

In [ ]:
# Calculate Avg gaps post release
post_long_window = post_long_window.sort_values(by=['subscription_id','usage_date'])
post_long_window['gaps'] = post_long_window.groupby('subscription_id')['usage_date']\
                                             .diff() \
                                             .dt.days \
                                             .astype('Int64')

avg_gaps = post_long_window.groupby('subscription_id')['gaps'] \
            .mean() \
            .reset_index(name='post_avg_gaps')

# Calculate usage metrics: total_usage, total_usage_duration, avg_daily_usage, avg_daily_usage_duration
total_usage = post_long_window.groupby('subscription_id')['usage_count'].sum() \
                               .reset_index(name='post_total_usage')

total_usage_duration = post_long_window.groupby('subscription_id')['usage_duration_secs'].sum() \
                                        .reset_index(name='post_total_usage_duration')

total_error_count = post_long_window.groupby('subscription_id')['error_count'].sum() \
                                        .reset_index(name='post_total_error_count')

post_metrics = post_usage_ai.merge(avg_gaps, on='subscription_id', how='left')
post_metrics = post_metrics.merge(total_usage, on='subscription_id', how='left') 
post_metrics = post_metrics.merge(total_usage_duration, on='subscription_id', how='left')
post_metrics = post_metrics.merge(total_error_count, on='subscription_id', how='left')

# Calculate averages
post_metrics['post_avg_daily_usage'] = (post_metrics['post_total_usage'] / post_metrics['post_distinct_usage_days']).round(2)
post_metrics['post_avg_daily_usage_duration'] = (post_metrics['post_total_usage_duration'] / post_metrics['post_distinct_usage_days']).round(2)
post_metrics['post_avg_daily_error_count'] = (post_metrics['post_total_error_count'] / post_metrics['post_distinct_usage_days']).round(2)

# Check for dupes, ensure post_metrics is the subscription_id level
print(f"Check for duplicates: {post_metrics.duplicated(subset='subscription_id').any()}")

subs = pre_metrics.merge(post_metrics, on='subscription_id', how='left')